<a href="https://colab.research.google.com/github/neenz16/HCI/blob/main/analysis_test_CS298.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file into a pandas dataframe
df = pd.read_csv('sortedoutputwithmessags.csv')

# Find the start and end rows for each trial
trial_starts = df[df['messages'].str.startswith('Display Trial')].index
trial_ends = df[df['messages'].str.contains('User Respnse')].index.tolist()
trial_ends.append(len(df)) # add last row index to the end of trial_ends

# Loop through each trial and calculate the mean pupil diameter for left and right eyes
for i in range(len(trial_starts)):
    trial = df.loc[trial_starts[i]:trial_ends[i]]
    left_mean = np.nanmean(trial['left.pupil.diameter'].fillna(0))
    right_mean = np.nanmean(trial['right.pupil.diameter'].fillna(0))
    last_row = trial.iloc[-1]
    response = last_row['messages'].split('User Respnse: ')[-1].strip()
    character2 = None
    response_split = response.split(',')

    for s in response_split:
        if 'character' in s:
            character = s.split(':')[-1].strip()

    image_name = None
    if len(response_split) > 1:
        image_name = response_split[1].split(':')[-1].strip()
    trial_number = response_split[0].split()[-1]

    # Find the row with the total time for this trial
    total_time_row = df.loc[trial_ends[i]+1:trial_ends[i+1]].loc[df['messages'].str.contains('Total Time:')]
    if len(total_time_row) > 0:
        # Extract the total time value from the messages column
        total_time_value = total_time_row['messages'].str.extract(r'^Total Time: (\d+\.\d+)').iloc[0, 0]
    else:
        total_time_value = np.nan

    print(f'Trial {i+1}: Response = {character}, Left mean pupil diameter = {left_mean}, Right mean pupil diameter = {right_mean}, Total Time = {total_time_value}')



subject data .mat files to CSV

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
# Use Matlab file to generate the csv file
dataset = ['ResultTable_sub1.csv', 'ResultTable_sub5.csv', 'ResultTable_sub7.csv',
           'ResultTable_sub8.csv', 'ResultTable_sub9.csv', 'ResultTable_sub10.csv',
           'ResultTable_sub12.csv', 'ResultTable_sub13.csv', 'ResultTable_sub14.csv',
           'ResultTable_sub16.csv', 'ResultTable_sub17.csv','ResultTable_sub18.csv',
           'ResultTable_sub19.csv', 'ResultTable_sub20.csv','ResultTable_sub21.csv']


index = 0
# initializing string
num_string = dataset[index]
# using re.findall()
# getting numbers from string
temp = re.findall(r'\d+', num_string)
sub_num = list(map(int, temp))
# print result
print(sub_num)


# Define variables for the output:
tr = []
response = []
pupil_left = []
rt = []
character_type_array = []
block = []
block_no = 0
subject = []

In [ ]:

for j in range(len(dataset)):
  count_FI = 0
  count_FOMI = 0

  # initializing string
  num_string = dataset[j]
  # using re.findall()
  # getting numbers from string
  temp = re.findall(r'\d+', num_string)
  sub_num = list(map(int, temp))

  # Read the CSV file into a pandas dataframe
  df = pd.read_csv(dataset[j])

  # Find the start and end rows for each trial
  trial_starts = df[df['message'].str.startswith('Display Trial')].index
  trial_ends = df[df['message'].str.contains('User Respnse')].index.tolist()
  trial_ends.append(len(df)) # add last row index to the end of trial_ends

#print('Subject, Block, Trial, FI Response, FI Left mean pupil diameter , FI Total Time, MI/FO Response, MI/FO Left mean pupil diameter , MI/FO Total Time ')

  # Loop through each trial and calculate the mean pupil diameter for left and right eyes
  for i in range(len(trial_starts)):
      trial = df.loc[trial_starts[i]:trial_ends[i]]
      left_mean = np.nanmean(trial['leftPupilDiameter'].fillna(0))
      #right_mean = np.nanmean(trial['right.pupil.diameter'].fillna(0))
      last_row = trial.iloc[-1]
      response = last_row['message'].split('User Respnse: ')[-1].strip()
      character2 = None
      response_split = response.split(',')

      for s in response_split:
          if 'character' in s:
              character = s.split(':')[-1].strip()
              # Get the response FI or MI/FO

      image_name = None
      if len(response_split) > 1:
          image_name = response_split[1].split(':')[-1].strip()
      trial_number = response_split[0].split()[-1]

      # Find the row with the total time for this trial
      total_time_row = df.loc[trial_ends[i]+1:trial_ends[i+1]].loc[df['message'].str.contains('Total Time:')]
      if len(total_time_row) > 0:
          # Extract the total time value from the messages column
          total_time_value = total_time_row['message'].str.extract(r'^Total Time: (\d+\.\d+)').iloc[0, 0]
      else:
          total_time_value = np.nan
      # print(f'{i+1},  {character}, {left_mean},{total_time_value}')

      tr.append(i+1)
      if 'FI' in character:
          character_type = "".join("FI")
      else:
          character_type = "".join('MI/FO')
      character_type_array.append(character_type)
      pupil_left.append(left_mean)
      rt.append(total_time_value)
      if (i+1) <= 10:
          block_no = 1
      elif (i+1) > 10 and (i+1) <= 20:
          block_no = 2
      else:
          block_no = 3
      block.append(block_no)
      subject.append(sub_num)

      #print(f'{i+1},  {character}, {character_type},  {left_mean,},{total_time_value}')




d = {'subject': subject, 'block': block, 'trial': tr, 'character_type': character_type_array, 'pupil_ave': pupil_left, 'rt':rt}
df_results = pd.DataFrame(data = d)
df_results.head(5)


In [ ]:
# Divide the data based on the blocks
df_block1 = df_results[df_results['block'] == 1]
df_block3 = df_results[df_results['block'] == 3]


In [ ]:
df_block1.describe()


In [ ]:
df_block3.describe()


In [ ]:
# count the responses:
df_block1_FI = df_block1[df_block1['character_type']== 'FI']

In [ ]:
# count the responses:
df_block3_FI = df_block3[df_block3['character_type']== 'FI']

In [ ]:
df_block1_FI.count()
df_block3_FI.count()

df_block1_FI['pupil_ave'].mean()
df_block3_FI['pupil_ave'].mean()


#df_block1_FI['rt'].mean()
#df_block3_FI['rt'].mean()

print('block 1 -> count FI:', df_block1_FI.count(), 'pupil_ave FI:', df_block1_FI['pupil_ave'].mean())
print('block 3 -> count FI:', df_block3_FI.count(), 'pupil_ave FI:', df_block3_FI['pupil_ave'].mean())

In [ ]:
# t-test:
from scipy.stats import ttest_rel
t_test_FI_count = ttest_rel(df_block1['pupil_ave'], df_block3['pupil_ave'])
print(f"P-Value: {round(t_test_FI_count.pvalue,4)}")



matlab coode to convert t.mat to output.xls with device time and messages with pupil dilation stuff

In [ ]:
# # % how to run
# # % 1) change the variable 'temp' below to the name of .mat file being read
# # % 2) change the variable 'len' below to the number of rows/columns of data
# # % 3) change the variable 'messages_length' to the number of messages in the
# # % .mat file
# # % 4) change the variable 'output_file' to the name of the output file
# # % 5) hit run at the top and let the script run completely
# # % 6) check to see that the data in the output csv matches the data in the
# # % .mat file

# # % change variable based on name of file being read
# temp = load('t.mat');

# # % change variable for loop length based on the number of rows/columns
# len = 11514;

# # % change variable based on number of messages
# messages_length = 206;

# # % change variable based on what you want to name output file
# output_file = 'output.csv'

# # % naming the header
# file = table([],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],'VariableNames',{ ...
#     'deviceTimeStamp', ...
#     'systemTimeStamp', ...
#     'messages', ...
#     'left.gazePoint.onDisplayArea x', ...
#     'left.gazePoint.onDisplayArea y', ...
#     'left.gazePoint.inUserCoords x', ...
#     'left.gazePoint.inUserCoords y', ...
#     'left.gazePoint.inUserCoords z', ...
#     'left.pupil.diameter', ...
#     'left.gazeOrigin.inUserCoords x', ...
#     'left.gazeOrigin.inUserCoords y', ...
#     'left.gazeOrigin.inUserCoords z', ...
#     'right.gazePoint.onDisplayArea x', ...
#     'right.gazePoint.onDisplayArea y', ...
#     'right.gazePoint.inUserCoords x', ...
#     'right.gazePoint.inUserCoords y', ...
#     'right.gazePoint.inUserCoords z', ...
#     'right.pupil.diameter', ...
#     'right.gazeOrigin.inUserCoords x', ...
#     'right.gazeOrigin.inUserCoords y,' ...
#     'right.gazeOrigin.inUserCoords z'});

# message_file = table([], 'VariableNames',{'systemTimeStamp'});

# # % variables for transposing
# a = transpose(temp.data.gaze.left.gazeOrigin.inUserCoords);
# b = transpose(temp.data.gaze.right.gazeOrigin.inUserCoords);

# c = transpose(temp.data.gaze.left.gazePoint.inUserCoords);
# d = transpose(temp.data.gaze.right.gazePoint.inUserCoords);

# x = transpose(temp.data.gaze.left.gazePoint.onDisplayArea);
# y = transpose(temp.data.gaze.right.gazePoint.onDisplayArea);

# # % for loop to iterate take each piece of data and write to a row, then
# # % clear array and do it again
# for i = 1:len
#     newline = {temp.data.gaze.deviceTimeStamp(i), ...
#         temp.data.gaze.systemTimeStamp(i), ...
#         0, ...
#         x(i), ...
#         x(i+len), ...
#         c(i), ...
#         c(i+len), ...
#         c(i+(len*2)), ...
#         temp.data.gaze.left.pupil.diameter(i), ...
#         a(i), ...
#         a(i+len), ...
#         a(i+(len*2)), ...
#         y(i), ...
#         y(i+len), ...
#         d(i), ...
#         d(i+len), ...
#         d(i+(len*2)), ...
#         temp.data.gaze.right.pupil.diameter(i), ...
#         b(i), ...
#         b(i+len), ...
#         b(i+(len*2))};
#     file = [file ; newline];
#     writetable(file,output_file);
#     disp(i)
# end

# # % for loop to append messages below
# for i = 1:messages_length
#     writecell([0, temp.messages(i), temp.messages(messages_length+i)],output_file,'WriteMode','append');
#     disp(i)
# end